# Analizamos los emails de cada empleado contenidos en el dataset ENRON

In [ ]:
# Se abre el dataset original y se crea un fichero en blanco con extensión .mbox (en este caso se llama enron.mbox)
# el cual será sobreescrito con los datos legibles del dataset original. 
import re
import email
from time import asctime
import os
import sys 
from dateutil.parser import parse

#Indicar la ruta donde se encuentra el dataset
MAILDIR= "C:\\Users\\......\\enron_mail_20150507" 

#Indicar la ruta donde se guardará el nuevo dataset
MBOX = "C:\\Users\\.......\\enron.mbox"

file = open(MBOX, 'w+')

for(root, dirs, file_names) in os.walk(MAILDIR):
       
        if root.split(os.sep)[-1].lower() != 'inbox':
            continue
            
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            message_text = open(file_path, errors = 'ignore').read()
            
            _from = re.search(r"From:([^\r\n]+)", message_text).groups()[0]
            _date=re.search(r"Date:([^\r\n]+)", message_text).groups()[0]
            
            _date = asctime(parse(_date).timetuple())
            msg = email.message_from_string(message_text)
            msg.set_unixfrom('From {0} {1}'.format(_from, _date))
            
            file.write(msg.as_string(unixfrom=True) + "\n\n")
        
file.close()

In [ ]:
#Convertimos el dataset enron.mbox en un diccionario [] (Pandas Dataframe) para poder leerlo como una tabla de datos. 
# A partir de esta transformación de datos recién se puede hacer el análisis de texto
#la ejecución tarda 3hr min aprox

import pandas as pd #pip install pandas
import mailbox

#Indicar la ruta donde se encuentra el nuevo dataset
MBOX = "C:\\Users\\......\\enron.mbox"
file = mailbox.mbox(MBOX)
mbox_dict = {}
for i, msg in enumerate(file):
    mbox_dict[i] = {}
    for header in msg.keys():
        mbox_dict[i][header] = msg[header]
        mbox_dict[i]['Body'] = msg.get_payload().replace('\n','').replace('\t','').replace('\r','').strip()
        
    df_original = pd.DataFrame.from_dict(mbox_dict, orient='index')

In [ ]:
df_original.head(2)

In [ ]:
#convertimos el df_original a .csv porque si tenemos que volver a este punto no hará falta esperar tanto tiempo de compilación
df_original.to_csv("C:\\Users\\.....indicar ruta......\\df_original1.csv", sep= '|')

In [ ]:
##Modifico el dataframe para quedarme solo con las columnas que necesitaré analizar
import pandas as pd
import csv
import string

ruta = "C:\\Users\\...\\...\\...copy.csv"
df_data = pd.read_csv(ruta, sep= '|', dtype='unicode')

#Es para cambiar los nombres de las columnas manteniendo el nombre de los indices
df_data.rename(columns={'Body':'BODY_MAIL','From':'EMPLOYEE','X-Origin':'EMPLOYEE1'}, inplace=True) 
#Creo una copia de los valores del From en otra columna para que luego este sea el índice
df_data['EMPLOYEE'] = df_data['EMPLOYEE1'] 
#indicamos que 'EMPLOYEE' es el nuevo índice del dataframe (df)
df_data.index = df_data['EMPLOYEE']
pd.set_option('max_colwidth',170) #para ampliar el tamaño de la columna body
# Quito las columnas que no necesitaré y me quedo solo con dos: la que contiene el nombre del empleado 
#del buzón de correo y el cuerpo del mensaje
cols_to_keep = ['EMPLOYEE1','BODY_MAIL']
df_data = df_data[cols_to_keep]

## convertir todo a minúsculas
df_data["EMPLOYEE1"] = df_data["EMPLOYEE1"].str.lower()

#Verifico listando las 5 primeras filas como ejemplo
df_data.head(3)

In [ ]:
#Esto es solo para comprobar la cantidad de veces que aparecen los índices en el dataframe (df) donde notamos que hay duplicados
#####df_data.pivot_table(index=['EMPLOYEE'], aggfunc='size')

#Combinando con el método .agg('sum') los valores del dataframe ('Body') por cada index duplicado que existe en el df
df_data = df_data.groupby(['EMPLOYEE1']).agg('sum')

#Como antes, esto es solo para comprobar que los índices no estén duplicados
df_data.pivot_table(index=['EMPLOYEE1'], aggfunc='size')

In [ ]:
##Para ver cómo va quedando el dataframe podemos ejecutar 
df_data.sample(2)
#len(df_data) #Comprobar la cant de filas

In [ ]:
#Función para reemplazar las URL y e-mails que contiene el texto "body_MAIL" del inbox de cada empleado, para luego poder quitarlas palabra
#que no nos aporta valor y que si quisiéramos analizarlas aparte las tenemos en las otras dos columnas emails y urls

import re
import string

def clean_str(text):
      
    text = re.sub('(?P<url>https?://[^\s]+)', ' 8 ', text) #quitar URL
    text = re.sub(r'[\w\-][\w\-\.]+@[\w\-][\w\-\.]+[a-zA-Z]{1,4}', ' 9 ', text) #quitar email adressess
    
    return text.strip().lower()

clean = lambda x: clean_str(x)

df_data = pd.DataFrame(df_data.BODY_MAIL.apply(clean).dropna()) #  #aplica los cambios y reemplaza los valores

In [ ]:
#comprobamos que las urls y los emails se han reemplazado con los números 8 y 9
df_data.BODY_MAIL.loc['lay-k'] 

In [ ]:
#Función para quitar signos de puntuación y dígitos sueltos
import re
import string

def clean_str1(text):
    
    text = re.sub("\b*\d\b* ", " ", text) ##quita los dígitos sueltos y que están entre palabras
    text = re.sub("\b\w\b ", " ", text) ##quita las letras sueltas
    text = re.sub(r"[%s]" % re.escape(string.punctuation), " ", text) # Quitar todos los signos de puntuación
      
    
    return text.strip().lower()

clean1 = lambda x: clean_str1(x)

df_data = pd.DataFrame(df_data.BODY_MAIL.apply(clean1).dropna()) #  #aplica los cambios y reemplaza los valores

df_data.BODY_MAIL.loc['lay-k'] 

In [ ]:
import re
import string

#función para quitar los espacios en blanco entre palabras y otros

def clean_str2(text):
    
    text = re.sub('(\d+)', '', text) #quita los dígitos sueltos y que están entre palabras
    text = re.sub('  +', ' ', text)  #quita espacios en blanco
    text = re.sub("\b*\d\b* ", " ", text)
    text = re.sub("\b\w\b", " ", text)
       
    return text

clean2 = lambda x: clean_str2(x)

df_data = pd.DataFrame(df_data.BODY_MAIL.apply(clean2).dropna()) #  #aplica los cambios y reemplaza los valores

df_data.BODY_MAIL.loc['lay-k']

In [ ]:
import re
import string

#limpiamos nuevamente el texto

def clean_str3(text):
    
    text = re.sub('(\d+)', '', text)
    text = re.sub("\b\w\b", " ", text)
        
    return text.strip().lower()

clean3 = lambda x: clean_str3(x)

df_data = pd.DataFrame(df_data.BODY_MAIL.apply(clean3).dropna()) #  #aplica los cambios y reemplaza los valores

df_data.BODY_MAIL.loc['lay-k'] #ESTO ES PARA COMPROBAR CÓMO VA QUEDANDO EL TEXTO QUE SE ANALIZARÁ

In [ ]:
#Agregamos algunas palabras que hayamos identificado que en el texto son insignificantes
import nltk
from nltk.corpus import stopwords

stop_words = ["thank", "per", "one", "fax", "aa", "abc", "ab", "ziff", "aaa", "aaron", "please", "zip", "image", "td","width", "would", "may", "click", "com", "us", "thanks", "let", "get", "font", "inc", "tr", "link", "error", "two","go","cc","href","id", "also","img", "pm", "alt","set","lt","nt","th","xls","much", "might","title","br","ff","doc","well","ees","already","de","ge","rm","st","sf","cbs","se","fyi","dec","end","arial","rr","en","cd","uk","tx","sw","div","jd","ng","bo","hp", "due"] + list(stopwords.words('english'))
df_data['BODY_MAIL'] = df_data['BODY_MAIL'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [ ]:
## Ya tenemos el corpus (datos limpios), lo pasaremos a pickle para luego poder analizarlo
df_data.to_pickle("C:\\Users\\.....\\......\\corpus_01.pkl")

In [ ]:
##Creamos la matriz document-term matrix usando CountVectorizer y quitamos los stopwords en inglés.

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=20)
data_cv = cv.fit_transform(df_data.BODY_MAIL)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = df_data.index
data_dtm.sample(3)

In [ ]:
#El dtm.pkl guarda el count vectorizer (cv)
data_dtm.to_pickle("C:\\Users\\...\\....\\dtm_01.pkl")

In [ ]:
# Leemos y transponemos el document-term matrix para luego poder contar las palabras más comunes sin perder los índices
import pandas as pd
import pickle

data = pd.read_pickle("C:\\Users\\....\\dtm_01.pkl")
data = data.transpose()
data.head(5)

In [13]:
#columns_names = data.columns.values
#print(columns_names)

In [3]:
# Creamos un diccionario con las 2000 palabras top más comunes que utiliza cada uno de los empleados en sus correos
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(2000)
    top_dict[c]= list(zip(top.index, top.values))

#top_dict #---- Esto es para imprimir todos los valores

In [ ]:
top_dict

In [ ]:
# Imprimimos el diccionario creado (solo con 500 palabras de las 2000), desde donde se pueden inferir activos intangibles por cada empleado
for employee, top_words in top_dict.items():
    print(employee)
    print(', '.join([word for word, count in top_words[0:200]]))
    print('----------------------------')

In [ ]:
# Colocamos todas las 2000 palabras del diccionario en una sola lista
from collections import Counter

words = []
for employee in data.columns:
    top = [word for (word, count) in top_dict[employee]]
    for t in top:
        words.append(t)
        
words

In [ ]:
# Luego, mostramos la cantidad de empledos que usan cada una de las palabras de la lista
Counter(words).most_common()

In [ ]:
#Cantidad de palabras
employees = data.columns.values

unique_list = []
for body in data.columns:
    uniques = data[body].to_numpy().nonzero()[0].size
    unique_list.append(uniques)

# Creamos un dataframe con la cantidad de palabras únicas
data_words = pd.DataFrame(list(zip(employees, unique_list)), columns=['Empleados','Palabras_únicas'])
data_unique_sort = data_words.sort_values(by='Palabras_únicas')
data_unique_sort

In [ ]:
#Muestro una gráfica de barras del número total de palabras únicas por empleado
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [116, 56]
plt.figure(figsize = (40, 50), facecolor = None)

y_pos = np.arange(len(data_words))

plt.subplot(1, 2, 1)
plt.barh(y_pos, data_unique_sort.Palabras_únicas, align='center')
plt.yticks(y_pos, data_unique_sort.Empleados, fontsize=20)
plt.title('Número Total de Palabras Únicas por Empleado', fontsize=20)

plt.tight_layout()

#Guardamos la gráfica como imagen
plt.savefig ("C:\\Users\...\\..\palabraUnica.png") 

plt.show()

In [ ]:
#Calculamos la cantidad total de palabras utilizadas por cada empleado
employees = data.columns.values

total_list = []
for body in data.columns:
    total = data[body].to_numpy().sum()
    total_list.append(total)

# Create a new dataframe that contains this unique word count
data_words = pd.DataFrame(list(zip(employees, total_list)), columns=['Empleados','Total_palabras'])
data_total_sort = data_words.sort_values(by='Total_palabras')
data_total_sort

In [ ]:
#Lo anterior lo mostramos en una gráfica de barras
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [116, 56]
plt.figure(figsize = (40, 50), facecolor = None)

y_pos = np.arange(len(data_words))

plt.subplot(1, 2, 1)
plt.barh(y_pos, data_total_sort.Total_palabras, align='center')
plt.yticks(y_pos, data_total_sort.Empleados, fontsize=20)
plt.title('Número Total de Palabras por Empleado', fontsize=20)

plt.tight_layout()

plt.savefig ("C:\\Users\\...\\...\\palabras.png") 

plt.show()

# Ahora mostramos en un gráfico wordcloud las palabras más usadas por cada empleado

In [ ]:
import pandas as pd
import pickle
#Leo el corpus q es el mismo q el df_data y ahora lo llamamos data_corpus para trabajar el wordcloud

data_corpus = pd.read_pickle("C:\\Users\\...\\....\\corpus_01.pkl")
data_corpus.head(2)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stop_words = set(STOPWORDS)

wc = WordCloud(height= 800, width=800, stopwords=stop_words, max_words=300, min_font_size = 25, background_color="white")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees1 = ['allen-p',	'arnold-j',	'arora-h',	'badeer-r',	'bailey-s',	'bass-e',	'baughman-d',	'baughman-e',	'beck-s',	'benson-r']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data1 = data[employees1] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data1.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees1[index], fontsize=30)
    
plt.savefig ("C:\\Users\\r...\\...\\...\\wc1.png")   
#plt.show()  #Ejecutar esto en caso se quiera mostrar e gráfico en este mismo notebook

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees2 = ['blair-l',	'brawner-s',	'buy-r',	'campbell-l',	'carson-m',	'cash-m',	'causholli-m',	'corman-s',	'crandell-s',	'cuilla-m']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data2 = data[employees2] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data2.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees2[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc2.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees3 = ['dasovich-j',	'davis-d',	'dean-c',	'delainey-d',	'derrick-j',	'donoho-l',	'donohoe-t',	'dorland-c',	'ermis-f',	'farmer-d']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data3 = data[employees3] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data3.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees3[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\....\\wc3.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees4 = ['fischer-m',	'forney-j',	'gang-l',	'gay-r',	'geaccone-t',	'germany-c',	'gilbertsmith-d',	'giron-d',	'griffith-j',	'grigsby-m']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data4 = data[employees4] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data4.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees4[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\....\\wc4.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees5 = ['haedicke-m',	'keavey-p',	'harris-s',	'hayslett-r',	'heard-m',	'hendrickson-s',	'hernandez-j',	'hodge-j',	'holst-k',	'horton-s']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data5 = data[employees5] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data5.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees5[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc5.png")   
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees6 = ['hyatt-k',	'jones-t',	'kaminski-v',	'kean-s',	'keiser-k',	'king-j',	'kitchen-l',	'kuykendall-t',	'lavorado-j',	'lavorato-j']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data6 = data[employees6] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data6.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees6[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc6.png")   
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees7 = ['lay-k',	'lenhart-m',	'lewis-a',	'lokay-m',	'lokey-t',	'lucci-p',	'luchi-p',	'maggi-m',	'mann-k',	'martin-t']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data7 = data[employees7] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data7.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees7[index], fontsize=30)
    
plt.savefig ("C:\\Users\\...\\....\\wc7.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees8 = ['may-l',	'mcconnell-m',	'mckay-b',	'mckay-j',	'mclaughlin-e',	'meyers-a',	'mims-p',	'love-p',	'motley-m',	'neal-s']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data8 = data[employees8] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data8.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees8[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc8.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees9 = ['nemec-g',	'panus-s',	'parks-j',	'mccarty-d',	'perlingiere-d',	'phanis-s',	'pimenov-v',	'platter-p',	'presto-k',	'mims-thurston-p']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data9 = data[employees9] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data9.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees9[index], fontsize=30)
    
plt.savefig ("C:\\Users\\.....\\.....\\wc9.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees10 = ['quigley-d',	'rapp-b',	'reitmeyer-j',	'richey-c',	'ring-a',	'pereira-s',	'rogers-b',	'ruscitti-k',	'sager-e',	'saibi-e']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data10 = data[employees10] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data10.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees10[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\......\\wc10.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees11 = ['salisbury-h',	'quenet-j',	'sanders-r',	'scholtes-d',	'schoolcraft-d',	'schwieger-j',	'scott-s',	'ring-r',	'semperger-c',	'shackleton-s']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data11 = data[employees11] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data11.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees11[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc11.png")   
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees12 = ['shankman-j',	'shapiro-r',	'shively-h',	'sanchez-m',	'skilling-j',	'slinger-r',	'smith-m',	'solberg-g',	'staab-t',	'steffes-j']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data12 = data[employees12] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data12.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees12[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc12.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees13 = ['stepenovitch-j',	'storey-g',	'sturm-f',	'swerzbin-m',	'taylor-m',	'tholt-j',	'thomas-p',	'townsend-j',	'tycholiz-b',	'ward-k']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data13 = data[employees13] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data13.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees13[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\....\\wc13.png")   
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees14 = ['watson-k',	'weldon-c',	'weldon-v',	'whalley-g',	'white-s',	'whitt-m',	'williams-j',	'williams-w3',	'wolfe-j',	'ybarbo-p']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data14 = data[employees14] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data14.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees14[index], fontsize=30)
    
plt.savefig ("C:\\Users\\....\\.....\\wc14.png")   
#plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,100), facecolor = None)
plt.tight_layout(pad=2)


employees15 = ['zipper-a',	'zufferli-j']
            
 
#Agrego la siguiente línea para quedarme con las columnas que necesito de la matriz

data15 = data[employees15] 
    
#Crea subplots por cada empleado

for index, body in enumerate(data15.columns):
    wc.generate(data_corpus.BODY_MAIL[body])
    
    plt.subplot(10, 2, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(employees15[index], fontsize=30)
    
plt.savefig ("C:\\Users\\.....\\.....\\wc15.png")   
#plt.show()